## My work for predicting housing prices TRAZZ PEPPER

This is my work for using a kaggle dataset and beging to try and understand the relationships between the feature varaibles and the final price of the house. I have gone through and commeted extensivly to try and explain what I am doing and why I am doing what I am doing.

In [54]:
# The main packages I used are pandas, numpy and sklearn.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_squared_log_error

In [55]:
# The data was originally in a csv (train.csv)
path = "train.csv"
df = pd.read_csv(path)

In [56]:
# This is the function for data conditioning, this allows me formate the data for both training and testing the same.
# The first part turns categorical variables into dummies. (boolean values)


def data_conditioning(dataframe):
    dataframe.set_index("Id", inplace=True)
    categorical_columns = dataframe.select_dtypes(include=["object"]).columns

    for col in categorical_columns:
        dataframe = dataframe.drop(
            [col],
            axis=1,
        ).join(pd.get_dummies(df[col], prefix=f"{col}"))

    # Feature Engineering stuff I am trying, they make new variables from certian features
    # Turns Overall Quality into boolean as Overall quality was a numerical categorical variable.
    dataframe["Qual_Very_Excellent"] = (dataframe["OverallQual"] == 10).astype(int)
    dataframe["Qual_Excellent"] = (dataframe["OverallQual"] == 9).astype(int)
    dataframe["Qual_Very_Good"] = (dataframe["OverallQual"] == 8).astype(int)
    dataframe["Qual_Good"] = (dataframe["OverallQual"] == 7).astype(int)
    dataframe["Qual_Above_Average"] = (dataframe["OverallQual"] == 6).astype(int)
    dataframe["Qual_Average"] = (dataframe["OverallQual"] == 5).astype(int)
    dataframe["Qual_Below_Average"] = (dataframe["OverallQual"] == 4).astype(int)
    dataframe["Qual_Fair"] = (dataframe["OverallQual"] == 3).astype(int)
    dataframe["Qual_Poor"] = (dataframe["OverallQual"] == 2).astype(int)
    dataframe["Qual_Very_Poor"] = (dataframe["OverallQual"] == 1).astype(int)
    # dataframe = dataframe.drop(["OverallQual"], axis=1)

    # Turns Overall Condition into boolean as Overall condition was a numerical categorical variable.
    dataframe["Cond_Very_Excellent"] = (dataframe["OverallCond"] == 10).astype(int)
    dataframe["Cond_Excellent"] = (dataframe["OverallCond"] == 9).astype(int)
    dataframe["Cond_Very_Good"] = (dataframe["OverallCond"] == 8).astype(int)
    dataframe["Cond_Good"] = (dataframe["OverallCond"] == 7).astype(int)
    dataframe["Cond_Above_Average"] = (dataframe["OverallCond"] == 6).astype(int)
    dataframe["Cond_Average"] = (dataframe["OverallCond"] == 5).astype(int)
    dataframe["Cond_Below_Average"] = (dataframe["OverallCond"] == 4).astype(int)
    dataframe["Cond_Fair"] = (dataframe["OverallCond"] == 3).astype(int)
    dataframe["Cond_Poor"] = (dataframe["OverallCond"] == 2).astype(int)
    dataframe["Cond_Very_Poor"] = (dataframe["OverallCond"] == 1).astype(int)
    # dataframe = dataframe.drop(["OverallCond"], axis=1)

    # Turns MSSubClass into boolean as MSSubClass was a numerical categorical variable.
    dataframe["1-STORY 1946 & NEWER ALL STYLES"] = (
        dataframe["MSSubClass"] == 20
    ).astype(int)
    dataframe["1-STORY 1945 & OLDER"] = (dataframe["MSSubClass"] == 30).astype(int)
    dataframe["1-STORY W/FINISHED ATTIC ALL AGES"] = (
        dataframe["MSSubClass"] == 40
    ).astype(int)
    dataframe["1-1/2 STORY - UNFINISHED ALL AGES"] = (
        dataframe["MSSubClass"] == 45
    ).astype(int)
    dataframe["1-1/2 STORY FINISHED ALL AGES"] = (dataframe["MSSubClass"] == 50).astype(
        int
    )
    dataframe["2-STORY 1946 & NEWER"] = (dataframe["MSSubClass"] == 60).astype(int)
    dataframe["2-STORY 1945 & OLDER"] = (dataframe["MSSubClass"] == 70).astype(int)
    dataframe["2-1/2 STORY ALL AGES"] = (dataframe["MSSubClass"] == 75).astype(int)
    dataframe["SPLIT OR MULTI-LEVEL"] = (dataframe["MSSubClass"] == 80).astype(int)
    dataframe["SPLIT FOYER"] = (dataframe["MSSubClass"] == 85).astype(int)
    dataframe["DUPLEX - ALL STYLES AND AGES"] = (dataframe["MSSubClass"] == 90).astype(
        int
    )
    dataframe["1-STORY PUD (Planned Unit Development) - 1946"] = (
        dataframe["MSSubClass"] == 120
    ).astype(int)
    dataframe["1-1/2 STORY PUD - ALL AGES"] = (dataframe["MSSubClass"] == 150).astype(
        int
    )
    dataframe["2-STORY PUD - 1946 & NEWER"] = (dataframe["MSSubClass"] == 160).astype(
        int
    )
    dataframe["PUD - MULTILEVEL - INCL SPLIT LEV/FOYER"] = (
        dataframe["MSSubClass"] == 180
    ).astype(int)
    dataframe["2 FAMILY CONVERSION - ALL STYLES AND AGES"] = (
        dataframe["MSSubClass"] == 190
    ).astype(int)
    dataframe = dataframe.drop(["MSSubClass"], axis=1)

    # Making some new features too see if they better correlated with the dataframe.
    # Square some of the values and see the results
    dataframe["TotalBsmtSF_squared_FE"] = dataframe["TotalBsmtSF"] ** 2
    dataframe["LotFrontage_squared_FE"] = dataframe["LotFrontage"] ** 2

    # Lots of different variables
    dataframe["TotalBsmtSF_GarageArea_FE"] = dataframe["TotalBsmtSF"] * df["GarageArea"]
    dataframe["YearBuilt_age_FE"] = 20234 - dataframe["YearBuilt"]
    dataframe["Total_Baths_FE"] = (
        dataframe["FullBath"] + (dataframe["HalfBath"] / 2) + dataframe["BsmtFullBath"]
    )
    dataframe["GarageArea_to_TotalArea_FE"] = dataframe["GarageArea"] / (
        dataframe["GrLivArea"] + dataframe["TotalBsmtSF"] + 1e-5
    )
    dataframe["GrLivArea_to_GarageArea_FE"] = dataframe["GrLivArea"] / (
        dataframe["GarageArea"] + 1e-5
    )
    dataframe["TotalBsmtSF_to_GarageArea_FE"] = dataframe["TotalBsmtSF"] / (
        dataframe["GarageArea"] + 1e-5
    )
    dataframe["Years_Since_Renovation_FE"] = 2024 - dataframe["YearRemodAdd"]
    dataframe["TotalRooms_FE"] = dataframe["TotRmsAbvGrd"] + (
        dataframe["BsmtFinType1_GLQ"] > 0
    ).astype(int)
    dataframe["LotArea_to_GarageCars_FE"] = dataframe["LotArea"] / (
        dataframe["GarageCars"] + 1e-5
    )
    dataframe["Total_SF_FE"] = (
        dataframe["GrLivArea"]
        + dataframe["TotalBsmtSF"]
        + dataframe["1stFlrSF"]
        + dataframe["2ndFlrSF"]
    )

    dataframe["Total_Bedrooms_FE"] = (
        dataframe["BedroomAbvGr"]
        + dataframe["BsmtFinType1_BLQ"]
        + (dataframe["BsmtFinType1_GLQ"] > 0).astype(int)
    )
    dataframe["Garage_SF_to_Total_SF_FE"] = dataframe["GarageArea"] / (
        dataframe["Total_SF_FE"] + 1e-5
    )
    dataframe["GrLivArea_to_LotArea_FE"] = dataframe["GrLivArea"] / (
        dataframe["LotArea"] + 1e-5
    )
    dataframe["GarageCars_to_TotalRooms_FE"] = dataframe["GarageCars"] / (
        dataframe["TotalRooms_FE"] + 1e-5
    )
    dataframe["Outdoor_Space_FE"] = (
        dataframe["WoodDeckSF"]
        + dataframe["OpenPorchSF"]
        + dataframe["EnclosedPorch"]
        + dataframe["ScreenPorch"]
    )
    # Fills missing data with median (better ways to do this on improvement)
    dataframe.fillna(dataframe.median(), inplace=True)

    dataframe.columns = dataframe.columns.str.replace(" ", "_")

    return dataframe

In [57]:
# Data contitions and fills with median for missing values, and then scales the data
df = data_conditioning(dataframe=df)


# These columns are not in the test dataset so drop them
columns_to_drop = [
    "Condition2_RRAe",
    "Condition2_RRAn",
    "Condition2_RRNn",
    "Electrical_Mix",
    "Exterior1st_ImStucc",
    "Exterior1st_Stone",
    "Exterior2nd_Other",
    "GarageQual_Ex",
    "Heating_Floor",
    "Heating_OthW",
    "HouseStyle_2.5Fin",
    "MiscFeature_TenC",
    "PoolQC_Fa",
    "RoofMatl_ClyTile",
    "RoofMatl_Membran",
    "RoofMatl_Metal",
    "RoofMatl_Roll",
    "Utilities_NoSeWa",
]

df = df.drop(columns=columns_to_drop)


# Separate features and target variable
X = df.drop(columns=["SalePrice"], axis=1)
y = df["SalePrice"]

In [58]:
# Correlation Matrix to choose which variables to focus on. (however, correlation matrix too big so output a file better)
correlation_matrix = df.corr()
saleprice_correlation = (
    correlation_matrix["SalePrice"].abs().sort_values(ascending=False)
)

# Keep the variabels with correlation of 0.3 or higher (gives enough variabels but not too many to avoid overfitting)
high_corr_vars_list = saleprice_correlation[saleprice_correlation > 0.1].index.tolist()

# Shortens the amount of variables to the ones with highest correlation.
high_corr_vars = df[high_corr_vars_list]
del high_corr_vars_list[0]
X = high_corr_vars.drop(columns=["SalePrice"], axis=1)
y = high_corr_vars["SalePrice"]

In [59]:
# Scales the data all the same
scaler = StandardScaler()
X_S = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# PCA feature selection if necessary, found that PCA feature selection was not that helpful.
# pca = PCA(n_components=0.95)
# X_pca = pca.fit_transform(X_S)


# Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X_S, y, test_size=0.2, random_state=1
)

# Basic Linear Regression Model to see how good we start and how to imporve.
reg = LinearRegression()


reg.fit(X_train, y_train)

print(f"The linear model fits to training data: {round(reg.score(X_train, y_train),3)}")
print(f"The linear model fits to testing data: {round(reg.score(X_test, y_test),3)}")
print(f"")

The linear model fits to training data: 0.91
The linear model fits to testing data: -8.364167523314642e+19



In [60]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

# Initialize a RandomForestRegressor with a fixed random seed for reproducibility
forest = RandomForestRegressor(random_state=1)

# Fit the model to the training data
forest.fit(X_train, y_train)

# Perform 10-fold cross-validation on the training data to assess model performance
cv_scores = cross_val_score(forest, X_train, y_train, cv=10)

# Calculate the mean and standard deviation of the cross-validation scores
mean_cv_score = np.mean(cv_scores)
std_cv_score = np.std(cv_scores)

# Print the mean cross-validation score with its standard deviation
print(f"Mean CV Score: {mean_cv_score:.4f} ± {std_cv_score:.4f}")

# Evaluate the model on the test data to check its performance on unseen data
test_score = forest.score(X_test, y_test)

# Print the test score (R² coefficient of determination)
print(f"Test Score: {test_score:.4f}")

msle = mean_squared_log_error(y_test, forest.predict(X_test))
neg_msle = -msle
print(f"Negative Mean Squared Log Error (Test Set): {neg_msle}")

KeyboardInterrupt: 

In [32]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import loguniform, uniform

# This sets up a grid search to see if I can optimize the forest regression further.
param_grid = {
    "n_estimators": np.arange(50, 500, 50),
    "max_depth": np.arange(10, 70, 6),
    "min_samples_split": np.arange(2, 6),
    "min_samples_leaf": np.arange(1, 5),
    "max_features": ["sqrt", "log2", None],
    "min_weight_fraction_leaf": loguniform(1e-6, 0.4),
    "max_leaf_nodes": np.arange(20, 600, 45),
    "n_jobs": [-1],
    "max_samples": loguniform(0.2, 1),
}

# Set up GridSearchCV
grid_search = RandomizedSearchCV(
    RandomForestRegressor(),
    param_grid,
    n_iter=150,
    scoring="neg_mean_squared_log_error",
    cv=5,
    refit=True,
    n_jobs=-1,
)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best Parameters: {best_params}")

# Evaluate on the test set
best_model_RF = grid_search.best_estimator_
test_score = best_model_RF.score(X_test, y_test)
print(f"Test Score: {test_score}")

msle = mean_squared_log_error(y_test, best_model_RF.predict(X_test))
neg_msle = -msle
print(f"Negative Mean Squared Log Error (Test Set): {neg_msle}")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Parameters: {'max_depth': 40, 'max_features': None, 'max_leaf_nodes': 380, 'max_samples': 0.8921109597395811, 'min_samples_leaf': 2, 'min_samples_split': 2, 'min_weight_fraction_leaf': 2.5455063205214674e-06, 'n_estimators': 450, 'n_jobs': -1}
Test Score: 0.8838342902816522
Negative Mean Squared Log Error (Test Set): -0.021848450919677645


In [33]:
# The next model is using XGBoost
from lightgbm import LGBMRegressor

param_grid = {
    "num_leaves": np.arange(32, 300, 7),
    "n_jobs": [-1],
    "reg_lambda": loguniform(1e-5, 4),
    "reg_alpha": loguniform(1e-5, 4),
    "n_estimators": np.arange(50, 341, 40),
    "learning_rate": loguniform(0.001, 0.2),
    "max_depth": np.arange(15, 55, 4),
    "min_split_gain": loguniform(1e-8, 0.1),
    "min_child_weight": loguniform(1e-4, 0.1),
    "min_child_samples": np.arange(14, 30, 4),
    "subsample": loguniform(0.25, 1),
    "colsample_bytree": loguniform(0.25, 1),
}

LGBM_search = RandomizedSearchCV(
    LGBMRegressor(verbose=-1),
    param_distributions=param_grid,
    n_iter=150,
    scoring="neg_mean_squared_log_error",
    cv=5,
    refit=True,
)

LGBM_search.fit(X_train, y_train)

# Best parameters and best score
best_params_LGBM = LGBM_search.best_params_
best_score_LGBM = LGBM_search.best_score_

print(f"Best Parameters: {best_params_LGBM}")

# Evaluate on the test set
best_model_LGBM = LGBM_search.best_estimator_
test_score = best_model_LGBM.score(X_test, y_test)
print(f"Test Score: {test_score}")

msle = mean_squared_log_error(y_test, best_model_LGBM.predict(X_test))
neg_msle = -msle
print(f"Negative Mean Squared Log Error (Test Set): {neg_msle}")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Parameters: {'colsample_bytree': 0.2665263052874788, 'learning_rate': 0.03317048763266522, 'max_depth': 51, 'min_child_samples': 26, 'min_child_weight': 0.030912154782621458, 'min_split_gain': 0.0010263731195722509, 'n_estimators': 250, 'n_jobs': -1, 'num_leaves': 95, 'reg_alpha': 0.00010145196742561309, 'reg_lambda': 0.001064197017530975, 'subsample': 0.45026047457274737}
Test Score: 0.8942504089735344
Negative Mean Squared Log Error (Test Set): -0.017783618537742196


In [34]:
# The next model is using CatBoost
from catboost import CatBoostRegressor

# Define parameter grid for CatBoost
param_dis = {
    "iterations": np.arange(750, 2500, 250),
    "learning_rate": loguniform(0.005, 0.1),
    "l2_leaf_reg": loguniform(0.8, 50),
    "bagging_temperature": loguniform(1e-4, 1),
    "random_strength": np.arange(1, 10),
    "depth": np.arange(6, 10),
    "min_data_in_leaf": np.arange(1, 4),
    "border_count": np.arange(24, 296, 50),
}

# Initialize the RandomizedSearchCV for CatBoost
catboost_search = RandomizedSearchCV(
    CatBoostRegressor(verbose=0),
    param_distributions=param_dis,
    n_iter=10,
    scoring="neg_mean_squared_log_error",
    cv=5,
    refit=True,
    n_jobs=-1,
)
# Fit the search model to training data
catboost_search.fit(X_train, y_train)

# Extract the best parameters and score
best_params_catboost = catboost_search.best_params_
best_score_catboost = catboost_search.best_score_

# Print best parameters and score
print(f"Best Parameters: {best_params_catboost}")

# Evaluate on the test set
best_model_catboost = catboost_search.best_estimator_
test_score_catboost = best_model_catboost.score(X_test, y_test)
print(f"Test Score: {test_score_catboost}")

# Calculate and print the Negative Mean Squared Log Error (MSLE)
msle_catboost = mean_squared_log_error(y_test, best_model_catboost.predict(X_test))
neg_msle_catboost = -msle_catboost
print(f"Negative Mean Squared Log Error (Test Set): {neg_msle_catboost}")

Best Parameters: {'bagging_temperature': 0.00011911065033602194, 'border_count': 74, 'depth': 6, 'iterations': 1000, 'l2_leaf_reg': 1.3259722318146878, 'learning_rate': 0.062428781201099126, 'min_data_in_leaf': 3, 'random_strength': 6}
Test Score: 0.8896658377220079
Negative Mean Squared Log Error (Test Set): -0.015974707396513348


In [43]:
from xgboost import XGBRegressor

# Define parameter grid for XGBoost
param_dis_xgb = {
    "n_estimators": np.arange(100, 1000, 100),  # Number of boosting rounds
    "learning_rate": loguniform(0.005, 0.1),  # Step size shrinkage
    "max_depth": np.arange(3, 12, 2),  # Maximum depth of a tree
    "min_child_weight": np.arange(
        1, 10
    ),  # Minimum sum of instance weight (hessian) needed in a child
    "gamma": loguniform(
        0.001, 5
    ),  # Minimum loss reduction required to make a further partition
    "subsample": loguniform(0.5, 1),  # Subsample ratio of the training instances
    "colsample_bytree": loguniform(
        0.5, 1
    ),  # Subsample ratio of columns when constructing each tree
    "reg_alpha": loguniform(1e-4, 1),  # L1 regularization term on weights
    "reg_lambda": loguniform(0.1, 10),  # L2 regularization term on weights
    "scale_pos_weight": np.arange(1, 10),  # Balancing of positive and negative weights
    "booster": ["gbtree", "dart"],  # Type of boosting to use
    "tree_method": [
        "auto",
        "exact",
        "approx",
        "hist",
        "gpu_hist",
    ],  # Algorithm to use for tree construction
    "grow_policy": [
        "depthwise",
        "lossguide",
    ],  # Controls how trees grow (depthwise or loss-based)
}


# Initialize RandomizedSearchCV for XGBoost
xgboost_search = RandomizedSearchCV(
    XGBRegressor(verbosity=0),  # verbosity=0 to silence logs
    param_distributions=param_dis_xgb,
    n_iter=50,  # Number of parameter settings sampled
    scoring="neg_mean_squared_log_error",
    cv=5,
    refit=True,
    n_jobs=-1,
)

# Fit the search model to training data
xgboost_search.fit(X_train, y_train)

# Extract the best parameters and score
best_params_xgb = xgboost_search.best_params_
best_score_xgb = xgboost_search.best_score_

# Print best parameters and score
print(f"Best Parameters: {best_params_xgb}")

# Evaluate on the test set
best_model_xgb = xgboost_search.best_estimator_
test_score_xgb = best_model_xgb.score(X_test, y_test)
print(f"Test Score: {test_score_xgb}")

# Calculate and print the Negative Mean Squared Log Error (MSLE)
msle_xgb = mean_squared_log_error(y_test, best_model_xgb.predict(X_test))
neg_msle_xgb = -msle_xgb
print(f"Negative Mean Squared Log Error (Test Set): {neg_msle_xgb}")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
50 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_scor

Best Parameters: {'booster': 'dart', 'colsample_bytree': 0.6697760014556138, 'gamma': 2.1504113834101006, 'grow_policy': 'depthwise', 'learning_rate': 0.0318845601377782, 'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 900, 'reg_alpha': 0.03247829314605702, 'reg_lambda': 2.1227113395883195, 'scale_pos_weight': 3, 'subsample': 0.5590005601823681, 'tree_method': 'approx'}
Test Score: 0.9150921106338501
Negative Mean Squared Log Error (Test Set): -0.016573631559783936


In [67]:
from sklearn.ensemble import VotingRegressor

# Create a voting regressor
voting_regressor = VotingRegressor(
    estimators=[
        ("catboost", best_model_catboost),
        ("xgboost", best_model_xgb),
        ("lightgbm", best_model_LGBM),
    ],
    weights=[10, 6, 3],  # Equal weights for each model
)

# Fit the voting regressor
voting_regressor.fit(X_train, y_train)

# Make predictions
voting_predictions = voting_regressor.predict(X_test)
voting_regressor_score = voting_regressor.score(X_test, y_test)
print(f"Test Score: {voting_regressor_score}")

# Evaluate
voting_msle = mean_squared_log_error(y_test, voting_predictions)
neg_voting_msle = -voting_msle
print(f"Negative Mean Squared Log Error (Voting Regressor): {neg_voting_msle}")

Test Score: 0.9027601906178491
Negative Mean Squared Log Error (Voting Regressor): -0.015520460935240362


In [64]:
from sklearn.ensemble import StackingRegressor

# Create a stacking regressor
stacking_regressor = StackingRegressor(
    estimators=[
        ("catboost", best_model_catboost),
        ("xgboost", best_model_xgb),
        ("lightgbm", best_model_LGBM),
    ],
    final_estimator=CatBoostRegressor(),  # You can choose any regressor as the final estimator
)

# Fit the stacking regressor
stacking_regressor.fit(X_train, y_train)

# Make predictions
stacking_predictions = stacking_regressor.predict(X_test)
stacking_regressor_score = stacking_regressor.score(X_test, y_test)
print(f"Test Score: {stacking_regressor_score}")

# Evaluate
stacking_msle = mean_squared_log_error(y_test, stacking_predictions)
neg_stacking_msle = -stacking_msle
print(f"Negative Mean Squared Log Error (Stacking Regressor): {neg_stacking_msle}")

Learning rate set to 0.04196
0:	learn: 75450.0380679	total: 1.32ms	remaining: 1.32s
1:	learn: 72995.5922483	total: 2.52ms	remaining: 1.25s
2:	learn: 70658.4475762	total: 3.6ms	remaining: 1.2s
3:	learn: 68401.6344151	total: 4.76ms	remaining: 1.18s
4:	learn: 66216.3987905	total: 5.75ms	remaining: 1.14s
5:	learn: 64098.4250183	total: 6.81ms	remaining: 1.13s
6:	learn: 62136.5139485	total: 7.95ms	remaining: 1.13s
7:	learn: 60277.3153204	total: 9.16ms	remaining: 1.14s
8:	learn: 58527.4080367	total: 10.2ms	remaining: 1.13s
9:	learn: 56783.0336274	total: 11.4ms	remaining: 1.13s
10:	learn: 55096.7010609	total: 12.5ms	remaining: 1.12s
11:	learn: 53495.0355482	total: 13.5ms	remaining: 1.11s
12:	learn: 51994.1841011	total: 14.8ms	remaining: 1.12s
13:	learn: 50558.0522463	total: 15.8ms	remaining: 1.11s
14:	learn: 49230.7822313	total: 16.7ms	remaining: 1.09s
15:	learn: 47960.5364602	total: 17.9ms	remaining: 1.1s
16:	learn: 46722.5181524	total: 19ms	remaining: 1.1s
17:	learn: 45529.7473170	total: 20.

In [69]:
# Here I read in the test file, condition the data and then predict the outomes to a csv file.
path = "test.csv"
df = pd.read_csv(path)
df_index = df.Id
used_model = voting_regressor

df_test = data_conditioning(dataframe=df)
df_test = df_test[high_corr_vars_list]
df_test_S = pd.DataFrame(scaler.transform(df_test), columns=df_test.columns)
# df_pca_test = pca.transform(df_test_S)

y_prediction = used_model.predict(df_test_S)

output_df = pd.DataFrame({"Id": df_index, "SalePrice": y_prediction})
output_df.set_index("Id").to_csv("Housing_Price_Estimates_TP.csv")

## Ideas moving forward
- Potential for multicolinearity (too many variables look to reduce some) more into feature reduction
- Have missing values, better way to estimate the values of those. (instead of median)
- Better feature engineering, (area per car), (other square footage), (total square footage), (remove some basement sq features), (year built/remodeling), (OverallQual + OverallCon merge), (lot area divided by lot frontage)
- Merged model, using catboost, xgboost, and LGBM
- Potential to feed that into nerual network to improve further?
Best so far: catboost Test Score: 0.8896658377220079
Negative Mean Squared Log Error (Test Set): -0.015974707396513348
xgboost
Test Score: 0.9150921106338501
Negative Mean Squared Log Error (Test Set): -0.016573631559783936
